## importações e configurações

In [ ]:
!pip uninstall -y prophet cmdstanpy pystan
!pip install -U prophet cmdstanpy

In [ ]:
import os, sys
from pathlib import Path
import importlib
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple, Optional, Any
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV, GridSearchCV

# modelos
from lightgbm import LGBMRegressor
import lightgbm as lgb

from prophet import Prophet

In [ ]:
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [ ]:
# Caminho raiz do projeto
PROJ = Path("/content/drive/MyDrive/tcc-modelo/tcc-demand-forecasting")

# monta o drive
from google.colab import drive
drive.mount('/content/drive')

# Garante que o PROJECT_DIR está no sys.path
if str(PROJ) not in sys.path:
    sys.path.append(str(PROJ))

print("Repositório ativo em:", PROJ)

In [ ]:
from src.evaluations.models_metrics import calculate_metrics, compare_models

interim_dir = PROJ / "data" / "interim"
output_name_imputed = "olist_weekly_agg_withlags_imputed_2.parquet"
df_path = interim_dir / output_name_imputed

In [ ]:
# Colunas do seu dataset
date_col    = "order_week"   # mesma coluna para carimbar previsões
target_col  = "sales_qty"    # alvo
id_col      = "id"   # opcional

# Períodos
train_start     = pd.Timestamp("2017-04-01")
first_train_end = pd.Timestamp("2018-03-18")
test_start      = pd.Timestamp("2018-03-19")
test_end        = pd.Timestamp("2018-07-31")

# Janela de rolling (ex.: blocos de 1 semana)
step = pd.Timedelta(days=7)  # (gap após cutoff, janela)

## definição do df

### meu df sem filtro

In [ ]:
output_name_imputed = "olist_weekly_agg_withlags_imputed.parquet"
df_path_all = interim_dir / output_name_imputed

In [ ]:
meu_df_all = pd.read_parquet(df_path_all)

In [ ]:
meu_df_all.shape

### meu df
- já está filtrado o top7 sp

In [ ]:
# fazer um teste com exatamente o mesmo dataframe utilizado por Feras

In [ ]:
df = pd.read_parquet(df_path)

In [ ]:
df = df[(df[date_col]>= train_start) & (df[date_col]<= test_end)]

In [ ]:
df['id'] = df.index

In [ ]:
deciles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
df['order_week'].describe(percentiles=deciles)

### df feras

In [ ]:
import pandas as pd

# 1. Carregar todos os CSVs RAW do Olist usando seu loader
from src.data.loader import load_dataset

cfg_path = "/content/drive/MyDrive/tcc-modelo/tcc-demand-forecasting/configs/data.yaml"
dfs = load_dataset(str(cfg_path), dataset="olist", stage="raw")

customers      = dfs["olist_customers_dataset.csv"]
products       = dfs["olist_products_dataset.csv"]
orders         = dfs["olist_orders_dataset.csv"]
order_items    = dfs["olist_order_items_dataset.csv"]
order_payments = dfs["olist_order_payments_dataset.csv"]
pc_name_trans  = dfs["product_category_name_translation.csv"]

# --- 2. Traduzir nomes de categoria para inglês (igual ao repo original) ---
pc_map = pc_name_trans.set_index("product_category_name")["product_category_name_english"].to_dict()
products["product_category_name"] = products["product_category_name"].map(pc_map)

# --- 3. Replicar os merges principais do projeto original ---
sales_order = orders.merge(customers, on="customer_id", how="inner")
sales_order_item = order_items.merge(sales_order, on="order_id", how="left")
sales_order_full = sales_order_item.merge(products, on="product_id", how="inner")
sales_with_payments = sales_order_full.merge(order_payments, on="order_id", how="inner")

# --- 4. Tratar datas e índice de tempo ---
sales_with_payments["order_approved_at"] = pd.to_datetime(
    sales_with_payments["order_approved_at"],
    errors="coerce"
)

# filtro de sp
#sales_with_payments = sales_with_payments[sales_with_payments['customer_state'] == 'SP']

# remover linhas sem data, senão o .resample quebra
sales_with_payments = sales_with_payments.dropna(subset=["order_approved_at"])

# definir índice de tempo
sales_with_payments = sales_with_payments.set_index("order_approved_at")

# --- 5. Agregar por semana e categoria (mesmo conceito do projeto original) ---
group_freq = "W"

df_olist_weekly = (
    sales_with_payments
      .groupby("product_category_name")
      .resample(group_freq)["payment_value"]
      .count()
      .reset_index()
)

# renomear colunas para o formato usado no Prophet
#df_olist_weekly = df_olist_weekly.rename(
#    columns={
#        "order_approved_at": "ds",
#        "payment_value": "y"
#    }
#)

# garantir ordenação
df_olist_weekly = df_olist_weekly.sort_values(["product_category_name", "order_approved_at"]).reset_index(drop=True)

df_olist_weekly.columns

## lista de features

In [ ]:
desconsiderar = [ 'approval_time_hours_mean_co',
 'approval_time_hours_mean_ne',
 'approval_time_hours_mean_n',
 'approval_time_hours_mean_se',
 'approval_time_hours_mean_s',
 'delivery_diff_estimated_mean_co',
 'delivery_diff_estimated_mean_ne',
 'delivery_diff_estimated_mean_n',
 'delivery_diff_estimated_mean_se',
 'delivery_diff_estimated_mean_s',
 'est_delivery_lead_days_mean_co',
 'est_delivery_lead_days_mean_ne',
 'est_delivery_lead_days_mean_n',
 'est_delivery_lead_days_mean_se',
 'est_delivery_lead_days_mean_s',
 'delivery_diff_estimated_weighted',
 'est_delivery_lead_days_weighted',
 'approval_time_hours_weighted',
 'customer_regions']

In [ ]:
# Garante tipo datetime
df[date_col] = pd.to_datetime(df[date_col])

# Seleciona features "completas" (a função já exclui y/id/date por padrão)
#features = feature_set_all(df.columns.tolist())
#features = [c for c in features if c not in desconsiderar]

In [ ]:
selected_features = ['sales_qty_roll8_mean',
 'sales_qty_lag1',
 'sales_qty_roll4_mean',
 'sales_qty_lag2',
 'sales_qty_lag4',
 'sales_qty_lag8',
 'sales_qty_roll8_std',
 'sales_qty_roll4_std',
 'approval_time_hours_weighted_roll8_std',
 'price_var_m4_vs_prev4_mean_roll8_std',
 'est_delivery_lead_days_weighted_roll8_std',
 'approval_time_hours_weighted_roll4_std',
 'est_delivery_lead_days_weighted',
 'delivery_diff_estimated_weighted',
 'est_delivery_lead_days_weighted_roll4_std',
 'price_var_m4_vs_prev4_mean_roll4_std',
 'price_var_w1_point_mean_roll8_std',
 'price_var_w1_point_mean_roll4_std',
 'price_var_w1_smooth_mean_roll8_std',
 'approval_time_hours_weighted']

## funcoes uteis

In [ ]:
# ---------- Esquemas de treinamento ----------
def split_rolling(
    df: pd.DataFrame,
    date_col: str,
    first_train_end: pd.Timestamp,
    step: pd.Timedelta,
) -> List[Tuple[pd.DataFrame, pd.DataFrame]]:
    """
    Expanding window simples: a cada iteração,
    - treino = tudo até current_end
    - validação = (current_end, current_end + step]
    Avança current_end em 'step' a cada loop.
    """
    pairs = []
    current_end = first_train_end

    while True:
        val_end = current_end + step

        train = df[df[date_col] <= current_end].copy()
        valid = df[(df[date_col] > current_end) & (df[date_col] <= val_end)].copy()

        if valid.empty:
            break

        pairs.append((train, valid))
        current_end = val_end  # avança a janela

    return pairs

# ---------- Gera dataframes pela categoria do produto ----------
def split_by_category(df: pd.DataFrame, category_col: str):
  """
  Retorna um dicionário de dataframes, sendo um para cada categoria de produto
  """
  dfs = {}
  for col in df[category_col].unique():
    dfs[col] = df[df[category_col] == col].copy()
  return dfs


In [ ]:
 # ---------- gráfico predito x real ----------
def plot_real_pred(real: np.array, pred: np.array):
  """
  Plota gráfico de linhas com valores reais vs o predito
  """

  plt.plot(y_true_cat[:50], label="true")
  plt.plot(y_pred_cat[:50], label="pred")
  plt.legend()
  plt.show()

In [ ]:
 # ---------- HPO estático ----------
def run_lgbm_hpo_static(
    df: pd.DataFrame,
    features: List[str],
    date_col: str,
    target_col: str,
    first_train_end: pd.Timestamp,
    n_splits: int = 3,
    n_iter: int = 30,
) -> Dict:
    """
    Faz HPO estático (uma vez só) usando todo o período até first_train_end.
    Retorna best_params para serem reutilizados nos backtests por categoria.
    """
    # Usa apenas dados até o fim do treino estático
    df_train = df[df[date_col] <= first_train_end].sort_values(date_col).copy()

    X = df_train[features]
    y = df_train[target_col]

    tscv = TimeSeriesSplit(n_splits=n_splits)

    # Espaço de busca reduzido para não explodir o tempo
    param = {
        "num_leaves": [31, 63, 127],
        "max_depth": [-1, 8, 12],
        "learning_rate": [0.03, 0.05, 0.1],
        "n_estimators": [200, 400, 800],
        "subsample": [0.8, 1.0],
        "colsample_bytree": [0.8, 1.0],
        "min_child_samples": [10, 20, 30],
        "reg_alpha": [0.0, 0.1],
        "reg_lambda": [0.0, 0.1],
    }

    base = LGBMRegressor(
        random_state=42,
        n_jobs=-1
    )

    searcher = RandomizedSearchCV(
        estimator=base,
        param_distributions=param,
        n_iter=n_iter,
        scoring="neg_root_mean_squared_error",
        cv=tscv,
        n_jobs=-1,
        verbose=1,
        random_state=42,
    )

    searcher.fit(X, y)

    best_params = searcher.best_params_
    print(">>> Best params (HPO estático):")
    print(best_params)

    return best_params

## modelos

### LGBM

#### função do modelo

In [ ]:
def fit_predict_lgbm_fixed(
    train: pd.DataFrame,
    valid: pd.DataFrame,
    features: List[str],
    date_col: str,
    target_col: str,
    best_params: Dict
):
    """
    Treina e prediz com LGBM usando hiperparâmetros fixos (best_params).
    """
    train_sorted = train.sort_values(date_col)
    valid_sorted = valid.sort_values(date_col)

    X_tr = train_sorted[features]
    y_tr = train_sorted[target_col]
    X_va = valid_sorted[features]
    y_va = valid_sorted[target_col]

    mdl = lgb.LGBMRegressor(
        random_state=42,
        **best_params
    )

    mdl.fit(
        X_tr, y_tr,
        eval_set=[(X_va, y_va)],
        eval_metric="smape",
        callbacks=[
            lgb.early_stopping(stopping_rounds=100, verbose=False)
        ]
    )

    preds = mdl.predict(X_va)
    return preds, mdl


#### params

In [ ]:
df_hpo = df[df[date_col] <= first_train_end]

In [ ]:
df_hpo_all = meu_df_all[meu_df_all[date_col] <= first_train_end]

In [ ]:


# ---- roda HPO uma vez só ----
best_params = run_lgbm_hpo_static(
    df=df_hpo_all,
    features=selected_features,
    date_col=date_col,
    target_col=target_col,
    first_train_end=first_train_end,
    n_splits=3,
    n_iter=50,
)

#### teste

In [ ]:
# Períodos  iguais ao de feras
train_start     = pd.Timestamp("2017-01-01")
first_train_end = pd.Timestamp("2018-01-06")
test_start      = pd.Timestamp("2018-01-07")
test_end        = pd.Timestamp("2018-08-12")

step = pd.Timedelta(days=7)  # (gap após cutoff, janela)

In [ ]:
results_by_cat = {}

for cat, df_cat in df.groupby("product_category_name"):
    print(f"\n######## Categoria: {cat} ########")

    # Gera splits rolling só para essa categoria
    pairs = split_rolling(
        df=df_cat,
        date_col=date_col,
        first_train_end=first_train_end,
        step=step,
    )

    y_true_cat = []
    y_pred_cat = []
    date = []

    for i, (train_i, valid_i) in enumerate(pairs):
        # limita a janela de validação ao período de teste global
        valid_i = valid_i[
            (valid_i[date_col] >= test_start) &
            (valid_i[date_col] <= test_end)
        ]
        if valid_i.empty:
            continue

        preds_i, _ = fit_predict_lgbm_fixed(
            train=train_i,
            valid=valid_i,
            features=selected_features,
            date_col=date_col,
            target_col=target_col,
            best_params=best_params,
        )

        y_true_cat.extend(valid_i[target_col].tolist())
        y_pred_cat.extend(preds_i.tolist())
        date.extend(valid_i[date_col].tolist())

    if len(y_true_cat) == 0:
        print(f"  >> Sem janelas válidas no período de teste para categoria {cat}, pulando.")
        continue

    y_true_cat = np.array(y_true_cat)
    y_pred_cat = np.array(y_pred_cat)

    results_by_cat[cat] = {
        "y_true": y_true_cat,
        "y_pred": y_pred_cat,
        "date": date
    }


In [ ]:
plot_real_pred(y_true_cat, y_pred_cat)

In [ ]:
# dataframe com os resultados de cada categoria
y_true_cat = []
y_pred_cat = []
date = []
categorias = []

for cat, results in results_by_cat.items():
    y_true_cat.extend(results['y_true'])
    y_pred_cat.extend(results['y_pred'])
    date.extend(results['date'])
    categorias.extend([cat] * len(results['y_true']))

df_all = pd.DataFrame({
    'y_true': y_true_cat,
    'y_pred': y_pred_cat,
    'date': date,
    'categoria': categorias
})

df_all

In [ ]:
# dataframe com as metricas de cada categoria e geral

metrics_all = calculate_metrics(df_all, 'y_true', 'y_pred').to_dict()
metrics_all['categoria'] = 'all'
metrics_all = pd.DataFrame([metrics_all])

for categoria in df_all['categoria'].unique():
  metrics_cat = calculate_metrics(df_all[df_all['categoria'] == categoria], 'y_true', 'y_pred').to_dict()
  metrics_cat['categoria'] = categoria
  metrics_cat = pd.DataFrame([metrics_cat])
  metrics_all = pd.concat([metrics_all, metrics_cat])

metrics_all

#### teste com todas categorias

In [ ]:
# Períodos  iguais ao de feras
train_start     = pd.Timestamp("2017-01-01")
first_train_end = pd.Timestamp("2018-01-06")
test_start      = pd.Timestamp("2018-01-07")
test_end        = pd.Timestamp("2018-08-12")

step = pd.Timedelta(days=7)  # (gap após cutoff, janela)

In [ ]:
results_by_cat = {}

for cat, df_cat in meu_df_all.groupby("product_category_name"):
    print(f"\n######## Categoria: {cat} ########")

    # Gera splits rolling só para essa categoria
    pairs = split_rolling(
        df=df_cat,
        date_col=date_col,
        first_train_end=first_train_end,
        step=step,
    )

    y_true_cat = []
    y_pred_cat = []
    date = []

    for i, (train_i, valid_i) in enumerate(pairs):
        # limita a janela de validação ao período de teste global
        valid_i = valid_i[
            (valid_i[date_col] >= test_start) &
            (valid_i[date_col] <= test_end)
        ]
        if valid_i.empty:
            continue

        preds_i, _ = fit_predict_lgbm_fixed(
            train=train_i,
            valid=valid_i,
            features=selected_features,
            date_col=date_col,
            target_col=target_col,
            best_params=best_params,
        )

        y_true_cat.extend(valid_i[target_col].tolist())
        y_pred_cat.extend(preds_i.tolist())
        date.extend(valid_i[date_col].tolist())

    if len(y_true_cat) == 0:
        print(f"  >> Sem janelas válidas no período de teste para categoria {cat}, pulando.")
        continue

    y_true_cat = np.array(y_true_cat)
    y_pred_cat = np.array(y_pred_cat)

    results_by_cat[cat] = {
        "y_true": y_true_cat,
        "y_pred": y_pred_cat,
        "date": date
    }


In [ ]:
# dataframe com os resultados de cada categoria
y_true_cat = []
y_pred_cat = []
date = []
categorias = []

for cat, results in results_by_cat.items():
    y_true_cat.extend(results['y_true'])
    y_pred_cat.extend(results['y_pred'])
    date.extend(results['date'])
    categorias.extend([cat] * len(results['y_true']))

df_all = pd.DataFrame({
    'y_true': y_true_cat,
    'y_pred': y_pred_cat,
    'date': date,
    'categoria': categorias
})

df_all

In [ ]:
# dataframe com as metricas de cada categoria e geral

metrics_all = calculate_metrics(df_all, 'y_true', 'y_pred').to_dict()
metrics_all['categoria'] = 'all'
metrics_all = pd.DataFrame([metrics_all])

for categoria in df_all['categoria'].unique():
  metrics_cat = calculate_metrics(df_all[df_all['categoria'] == categoria], 'y_true', 'y_pred').to_dict()
  metrics_cat['categoria'] = categoria
  metrics_cat = pd.DataFrame([metrics_cat])
  metrics_all = pd.concat([metrics_all, metrics_cat])

metrics_all

### Prophet

#### funcao do modelo

In [ ]:
from typing import List, Dict, Optional, Any
import pandas as pd
from prophet import Prophet


def fit_predict_prophet_fixed(
    train: pd.DataFrame,
    valid: pd.DataFrame,
    date_col: str,
    target_col: str,
    features: Optional[List[str]],
):
    """
    Treina e prediz com Prophet usando hiperparâmetros fixos (params).

    - date_col: coluna de datas
    - target_col: coluna target (ex.: 'sales_qty')
    - features: lista de colunas usadas como regressoras externas (podem ser [] ou None)
    - params: dicionário com hiperparâmetros do Prophet
      (ex.: {'seasonality_mode': 'additive', 'changepoint_prior_scale': 0.05, ...})
    """
    # Garante ordem temporal
    train_sorted = train.sort_values(date_col).copy()
    valid_sorted = valid.sort_values(date_col).copy()

    if features is None:
        features = []

    # Monta dataframes no formato esperado pelo Prophet (ds, y, + regressors)
    df_tr = train_sorted[[date_col, target_col] + features].rename(
        columns={date_col: "ds", target_col: "y"}
    )
    df_va = valid_sorted[[date_col] + features].rename(
        columns={date_col: "ds"}
    )

    # Instancia o modelo com os hiperparâmetros informados
    mdl = Prophet(weekly_seasonality=True,
                            yearly_seasonality=True,
                            daily_seasonality=False)

    # Adiciona regressoras externas, se houver
    for reg in features:
        mdl.add_regressor(reg)

    # Treina
    mdl.fit(df_tr)

    # Faz previsão para o período de validação
    forecast = mdl.predict(df_va)

    # Prophet devolve várias colunas; usamos yhat como predição
    preds = forecast["yhat"].values

    return preds, mdl


#### definindo parametros

In [ ]:
param_space = {
    "seasonality_mode": ["additive", "multiplicative"],
    "changepoint_prior_scale": [0.001, 0.01, 0.05, 0.1],
    "seasonality_prior_scale": [1.0, 5.0, 10.0],
    "weekly_seasonality": [True, False],
    "yearly_seasonality": [False, True],
    "daily_seasonality": [False],  # fixo
}


In [ ]:
def random_search_prophet_metrics(
    train: pd.DataFrame,
    valid: pd.DataFrame,
    date_col: str,
    target_col: str,
    features: Optional[List[str]],
    param_space: Dict[str, List[Any]],
    n_iter: int = 20,
    random_state: int = 42,
):
    """
    Faz random search simples de hiperparâmetros do Prophet e retorna
    um DataFrame com métricas + parâmetros de cada tentativa.

    - param_space: dict com lista de valores possíveis para cada hiperparâmetro
      ex.: {"changepoint_prior_scale": [0.01, 0.05], "seasonality_mode": ["additive", "multiplicative"], ...}
    - n_iter: número de amostras aleatórias de combinações de parâmetros
    """

    rng = np.random.default_rng(random_state)
    results = []

    # y_true da base de validação (df completo, sem separar categorias)
    y_true = valid[target_col].values

    for i in range(n_iter):
        # Sorteia uma combinação de parâmetros
        params = {
            name: rng.choice(values)
            for name, values in param_space.items()
        }

        # Treina e prediz
        preds, _ = fit_predict_prophet_fixed(
            train=train,
            valid=valid,
            date_col=date_col,
            target_col=target_col,
            features=features,
            params=params,
        )

        # Monta df de avaliação compatível com calculate_metrics()
        df_eval = valid[[date_col, target_col]].copy()
        df_eval = df_eval.rename(columns={target_col: "y_true"})
        df_eval["y_pred"] = preds

        # Calcula métricas
        metrics_dict = calculate_metrics(df_eval, "y_true", "y_pred").to_dict()

        # Adiciona info de parâmetros e iteração
        metrics_dict.update(params)
        metrics_dict["iter"] = i + 1

        results.append(metrics_dict)

    # DataFrame com uma linha por combinação de parâmetros
    results_df = pd.DataFrame(results)

    return results_df


In [ ]:
train_static = df[df[date_col] <= first_train_end].sort_values(date_col).copy()
test_static = df[df[date_col] > first_train_end].sort_values(date_col).copy()

In [ ]:
results_df = random_search_prophet_metrics(
    train=train_static,      # df de treino completo
    valid=test_static,       # df de validação completo
    date_col=date_col,  # exemplo
    target_col=target_col,
    features=None,           # ou lista de regressoras, se quiser testar
    param_space=param_space,
    n_iter=80,
    random_state=42,
)

In [ ]:
results_df.sort_values("wape").head()

#### teste

In [ ]:
results_by_cat = {}

for cat, df_cat in df.groupby("product_category_name"):
    print(f"\n######## Categoria: {cat} ########")

    # Gera splits rolling só para essa categoria
    pairs = split_rolling(
        df=df_cat,
        date_col=date_col,
        first_train_end=first_train_end,
        step=step,
    )

    y_true_cat = []
    y_pred_cat = []
    date = []

    for i, (train_i, valid_i) in enumerate(pairs):
        # limita a janela de validação ao período de teste global
        valid_i = valid_i[
            (valid_i[date_col] >= test_start) &
            (valid_i[date_col] <= test_end)
        ]
        if valid_i.empty:
            continue

        preds_i, _ = fit_predict_prophet_fixed(
            train=train_i,
            valid=valid_i,
            features=[],
            date_col=date_col,
            target_col=target_col
        )

        y_true_cat.extend(valid_i[target_col].tolist())
        y_pred_cat.extend(preds_i.tolist())
        date.extend(valid_i[date_col].tolist())

    if len(y_true_cat) == 0:
        print(f"  >> Sem janelas válidas no período de teste para categoria {cat}, pulando.")
        continue

    y_true_cat = np.array(y_true_cat)
    y_pred_cat = np.array(y_pred_cat)

    results_by_cat[cat] = {
        "y_true": y_true_cat,
        "y_pred": y_pred_cat,
        "date": date
    }

In [ ]:
# dataframe com os resultados de cada categoria
y_true_cat = []
y_pred_cat = []
date = []
categorias = []

for cat, results in results_by_cat.items():
    y_true_cat.extend(results['y_true'])
    y_pred_cat.extend(results['y_pred'])
    date.extend(results['date'])
    categorias.extend([cat] * len(results['y_true']))

df_all = pd.DataFrame({
    'y_true': y_true_cat,
    'y_pred': y_pred_cat,
    'date': date,
    'categoria': categorias
})

In [ ]:
# dataframe com as metricas de cada categoria e geral

metrics_all = calculate_metrics(df_all, 'y_true', 'y_pred').to_dict()
metrics_all['categoria'] = 'all'
metrics_all = pd.DataFrame([metrics_all])

for categoria in df_all['categoria'].unique():
  metrics_cat = calculate_metrics(df_all[df_all['categoria'] == categoria], 'y_true', 'y_pred').to_dict()
  metrics_cat['categoria'] = categoria
  metrics_cat = pd.DataFrame([metrics_cat])
  metrics_all = pd.concat([metrics_all, metrics_cat])

metrics_all

#### recriando o teste de feras

In [ ]:
import os
from time import time
from datetime import timedelta
import numpy as np
import pandas as pd

from prophet import Prophet  # ou: from fbprophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt

# -------------------------------------------------------
# 1) Definições de datas e categorias (params.yml)
# -------------------------------------------------------
experiment_dates = {
    "train_start": "2017-01-01",
    "test_start": "2018-01-07",
    "test_end":   "2018-08-12",
}

product_categories = [
    "bed_bath_table",
    "health_beauty",
    "sports_leisure",
    "furniture_decor",
    "housewares",
    "watches_gifts",
    "telephony",
]

# -------------------------------------------------------
# 2) Função make_dates (igual ao utils.py do repo)
# -------------------------------------------------------
def make_dates(experiment_dates: dict) -> pd.DataFrame:
    """
    Cria folds de datas com {train, valid, test}, com janelas de 4 semanas.
    Mesma lógica do src/utils.py do projeto original.
    """
    date_ranges = []
    i = 0

    while True:
        train_start_0 = pd.to_datetime(experiment_dates["train_start"])
        test_start_0  = pd.to_datetime(experiment_dates["test_start"])
        test_end_0    = pd.to_datetime(experiment_dates["test_end"])

        # deslocamento do fold em blocos de 4 semanas
        offset = timedelta(weeks=4 * i)

        train_start = train_start_0 + offset
        # train_end = test_start + offset - 1 semana - 4 semanas
        train_end   = test_start_0 + offset - timedelta(weeks=1) - timedelta(weeks=4)

        valid_start = test_start_0 + offset - timedelta(weeks=4)
        valid_end   = test_start_0 + offset - timedelta(weeks=1)

        test_start  = test_start_0 + offset
        # test_end = test_start_0 + (4*i + 4) semanas - 1 semana
        test_end    = test_start_0 + timedelta(weeks=(4 * i + 4)) - timedelta(weeks=1)

        dates_ = {
            "train_start": train_start,
            "train_end":   train_end,
            "valid_start": valid_start,
            "valid_end":   valid_end,
            "test_start":  test_start,
            "test_end":    test_end,
        }
        date_ranges.append(dates_)

        if test_end >= test_end_0:
            break
        i += 1

    return pd.DataFrame(date_ranges)

# -------------------------------------------------------
# 3) Métricas (baseadas em src/metrics.py, sem mlflow)
# -------------------------------------------------------
def mean_absolute_percentage_error(y_true, y_pred):
    # versão do projeto: soma 1 para evitar divisão por 0
    y_true = np.array(y_true) + 1
    y_pred = np.array(y_pred) + 1
    return np.mean(np.abs((y_true - y_pred) / y_true))


def weighted_mean_absolute_error(test, predict):
    # versão do projeto: soma 1, pondera pelo valor de teste
    test = np.array(test) + 1
    predict = np.array(predict) + 1
    fenmu = max(test)
    rs = []
    for i in range(len(test)):
        if test[i] == 0:
            p = 1
        else:
            p = test[i]
        fenzi = (abs(test[i] - predict[i])) * p * p
        rs.append(float(fenzi) / fenmu)
    return np.mean(rs)


def get_metrics(y_true, y_pred):
    wape = weighted_mean_absolute_error(y_true, y_pred)
    rmse = sqrt(mean_squared_error(y_true, y_pred))
    r2_metric = r2_score(y_true, y_pred)
    mape_metric = mean_absolute_percentage_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)

    return {
        "wape": wape,
        "rmse": rmse,
        "r2":   r2_metric,
        "mape": mape_metric,
        "mae":  mae,
    }

# -------------------------------------------------------
# 4) Geração dos folds de datas
# -------------------------------------------------------
date_ranges = make_dates(experiment_dates)

# -------------------------------------------------------
# 5) Garantir que merged_data está no formato certo
#    (ajuste aqui se seu df tiver outro nome)
# -------------------------------------------------------
# merged_data deve ter colunas:
#   - 'order_approved_at' (datetime semanal)
#   - 'product_category_name'
#   - 'payment_value'
# Se ainda não for datetime:
df_olist_weekly["order_approved_at"] = pd.to_datetime(df_olist_weekly["order_approved_at"])

# opcional: garantir ordenação global
merged_data = df_olist_weekly.sort_values(["product_category_name", "order_approved_at"]).reset_index(drop=True)

# -------------------------------------------------------
# 6) Loop principal do experimento Prophet (sem mlflow)
# -------------------------------------------------------
output_dir = "artifacts/predictions_prophet_original"
os.makedirs(output_dir, exist_ok=True)

all_results = []  # para guardar métricas por categoria

for prod_cat in product_categories:
    print(f"\n==============================")
    print(f"Processando categoria: {prod_cat}")
    print(f"==============================")

    start_timer = time()
    all_predictions = []

    # loop sobre os folds de tempo
    for row in date_ranges.itertuples(index=False):
        train_start, train_end, valid_start, valid_end, test_start, test_end = (
            row.train_start,
            row.train_end,
            row.valid_start,
            row.valid_end,
            row.test_start,
            row.test_end,
        )

        print(f"  Faixa: {train_start.date()} até {test_end.date()}")

        # Treino = [train_start, valid_end]
        train_mask = (
            (merged_data["order_approved_at"] >= train_start)
            & (merged_data["order_approved_at"] <= valid_end)
            & (merged_data["product_category_name"] == prod_cat)
        )
        train_x = merged_data.loc[train_mask, ["order_approved_at", "payment_value"]]

        # Teste = [test_start, test_end]
        test_mask = (
            (merged_data["order_approved_at"] >= test_start)
            & (merged_data["order_approved_at"] <= test_end)
            & (merged_data["product_category_name"] == prod_cat)
        )
        test_y = merged_data.loc[test_mask, ["order_approved_at", "payment_value"]]

        # renomear colunas para Prophet
        train_x = train_x.rename(columns={"order_approved_at": "ds", "payment_value": "y"})
        test_y  = test_y.rename(columns={"order_approved_at": "ds", "payment_value": "y"})

        # garantir ordenação temporal
        train_x = train_x.sort_values("ds").reset_index(drop=True)
        test_y  = test_y.sort_values("ds").reset_index(drop=True)

        # --------------------------------------------------
        # Rolling dentro do período de teste (igual ao código original)
        # --------------------------------------------------
        predictions = []
        for i in range(test_y.shape[0]):
            # modelo Prophet novo a cada passo
            model = Prophet(
                weekly_seasonality=True,
                yearly_seasonality=True,
                daily_seasonality=False,
            )
            model.add_country_holidays(country_name="BR")

            # concatena treino "fixo" com parte já observada do teste
            # (train_x.iloc[i:] vem do código original, apesar de estranho)
            df_fit = pd.concat([train_x.iloc[i:], test_y.iloc[:i]]).reset_index(drop=True)

            model.fit(df_fit)

            # previsão 1 passo à frente (7 dias, pois série é semanal)
            future = model.make_future_dataframe(periods=1, freq="7D")
            fcst = model.predict(future)["yhat"].iloc[-1]
            predictions.append(fcst)

        all_predictions.extend(predictions)

    # -------------------------------------------------------
    # Após todos os folds, calcular métricas no período de teste global
    # -------------------------------------------------------
    global_test_mask = (
        (merged_data["product_category_name"] == prod_cat)
        & (merged_data["order_approved_at"] >= pd.to_datetime(experiment_dates["test_start"]))
        & (merged_data["order_approved_at"] <= pd.to_datetime(experiment_dates["test_end"]))
    )

    df_filtered = merged_data.loc[global_test_mask, ["order_approved_at", "payment_value"]].copy()
    df_filtered = df_filtered.sort_values("order_approved_at").reset_index(drop=True)

    # garantir alinhamento: len(y_true) == len(all_predictions)
    y_true = df_filtered["payment_value"].values
    y_pred = np.array(all_predictions[: len(y_true)])  # safety

    metrics = get_metrics(y_true, y_pred)
    duration_min = int((time() - start_timer) // 60)

    print(f"  Métricas para {prod_cat}: {metrics}")
    print(f"  Tempo (min): {duration_min}")

    # salvar previsões em CSV (similar ao original)
    save_data = pd.DataFrame(
        {
            "y_true": y_true,
            "preds": y_pred,
            "dates": df_filtered["order_approved_at"],
        }
    )

    fname = os.path.join(output_dir, f"exp1_prophet_{prod_cat}.csv")
    save_data.to_csv(fname, index=False)

    # acumular resultado geral
    all_results.append(
        {
            "product_category": prod_cat,
            "time_min": duration_min,
            **metrics,
        }
    )

# -------------------------------------------------------
# 7) DataFrame consolidado de métricas por categoria
# -------------------------------------------------------
df_results = pd.DataFrame(all_results)
df_results


#### aplicando base de feras com meu pipeline

In [ ]:
experiment_dates = {
    "train_start": "2017-01-01",
    "test_start": "2018-01-07",
    "test_end":   "2018-08-12",
}

In [ ]:
# Períodos  iguais ao de feras
train_start     = pd.Timestamp("2017-01-01")
first_train_end = pd.Timestamp("2018-01-06")
test_start      = pd.Timestamp("2018-01-07")
test_end        = pd.Timestamp("2018-08-12")

step = pd.Timedelta(days=7)  # (gap após cutoff, janela)

In [ ]:
# meus periodos
train_start     = pd.Timestamp("2017-04-01")
first_train_end = pd.Timestamp("2018-03-18")
test_start      = pd.Timestamp("2018-03-19")
test_end        = pd.Timestamp("2018-07-31")

In [ ]:
product_categories = [
    "bed_bath_table",
    "health_beauty",
    "sports_leisure",
    "furniture_decor",
    "housewares",
    "watches_gifts",
    "telephony",
]

In [ ]:
df_olist_weekly = df_olist_weekly.rename(columns={"order_approved_at": "order_week", "payment_value": "sales_qty"})

In [ ]:
df_olist_weekly = df_olist_weekly[df_olist_weekly['product_category_name'].isin(product_categories)]

In [ ]:
results_by_cat = {}

for cat, df_cat in df_olist_weekly.groupby("product_category_name"):
    print(f"\n######## Categoria: {cat} ########")

    # Gera splits rolling só para essa categoria
    pairs = split_rolling(
        df=df_cat,
        date_col=date_col,
        first_train_end=first_train_end,
        step=step,
    )

    y_true_cat = []
    y_pred_cat = []
    date = []

    for i, (train_i, valid_i) in enumerate(pairs):
        # limita a janela de validação ao período de teste global
        valid_i = valid_i[
            (valid_i[date_col] >= test_start) &
            (valid_i[date_col] <= test_end)
        ]
        if valid_i.empty:
            continue

        preds_i, _ = fit_predict_prophet_fixed(
            train=train_i,
            valid=valid_i,
            features=[],
            date_col=date_col,
            target_col=target_col
        )

        y_true_cat.extend(valid_i[target_col].tolist())
        y_pred_cat.extend(preds_i.tolist())
        date.extend(valid_i[date_col].tolist())

    if len(y_true_cat) == 0:
        print(f"  >> Sem janelas válidas no período de teste para categoria {cat}, pulando.")
        continue

    y_true_cat = np.array(y_true_cat)
    y_pred_cat = np.array(y_pred_cat)

    results_by_cat[cat] = {
        "y_true": y_true_cat,
        "y_pred": y_pred_cat,
        "date": date
    }

In [ ]:
# dataframe com os resultados de cada categoria
y_true_cat = []
y_pred_cat = []
date = []
categorias = []

for cat, results in results_by_cat.items():
    y_true_cat.extend(results['y_true'])
    y_pred_cat.extend(results['y_pred'])
    date.extend(results['date'])
    categorias.extend([cat] * len(results['y_true']))

df_all = pd.DataFrame({
    'y_true': y_true_cat,
    'y_pred': y_pred_cat,
    'date': date,
    'categoria': categorias
})

In [ ]:
# dataframe com as metricas de cada categoria e geral

metrics_all = calculate_metrics(df_all, 'y_true', 'y_pred').to_dict()
metrics_all['categoria'] = 'all'
metrics_all = pd.DataFrame([metrics_all])

for categoria in df_all['categoria'].unique():
  metrics_cat = calculate_metrics(df_all[df_all['categoria'] == categoria], 'y_true', 'y_pred').to_dict()
  metrics_cat['categoria'] = categoria
  metrics_cat = pd.DataFrame([metrics_cat])
  metrics_all = pd.concat([metrics_all, metrics_cat])

metrics_all

#### aplicando para o dataset inteiro

In [ ]:
# Períodos  iguais ao de feras
train_start     = pd.Timestamp("2017-01-01")
first_train_end = pd.Timestamp("2018-01-06")
test_start      = pd.Timestamp("2018-01-07")
test_end        = pd.Timestamp("2018-08-12")

step = pd.Timedelta(days=7)  # (gap após cutoff, janela)

In [ ]:
df_olist_weekly = df_olist_weekly.rename(columns={"order_approved_at": "order_week", "payment_value": "sales_qty"})

In [ ]:
# agrupando por data, somando as vendas, sem categoria
df_olist_weekly = df_olist_weekly.groupby('order_week')['sales_qty'].sum().reset_index()

In [ ]:
df_olist_weekly.tail(5)

In [ ]:
x, y = pairs[1]

In [ ]:
x['product_category_name'].unique()

In [ ]:
pairs = split_rolling(
    df=df_olist_weekly,
    date_col=date_col,
    first_train_end=first_train_end,
    step=step,
)

y_true_cat = []
y_pred_cat = []
date = []

for i, (train_i, valid_i) in enumerate(pairs):
    # limita a janela de validação ao período de teste global
    valid_i = valid_i[
        (valid_i[date_col] >= test_start) &
        (valid_i[date_col] <= test_end)
    ]
    if valid_i.empty:
        continue

    preds_i, _ = fit_predict_prophet_fixed(
        train=train_i,
        valid=valid_i,
        features=[],
        date_col=date_col,
        target_col=target_col
    )

    y_true_cat.extend(valid_i[target_col].tolist())
    y_pred_cat.extend(preds_i.tolist())
    date.extend(valid_i[date_col].tolist())

y_true_cat = np.array(y_true_cat)
y_pred_cat = np.array(y_pred_cat)

results = pd.DataFrame({
    "y_true": y_true_cat,
    "y_pred": y_pred_cat,
    "date": date
})

In [ ]:
# dataframe com as metricas de cada categoria e geral

metrics_all = calculate_metrics(results, 'y_true', 'y_pred').to_dict()
metrics_all['categoria'] = 'all'
metrics_all = pd.DataFrame([metrics_all])


metrics_all